<a href="https://colab.research.google.com/github/hyunjaecho94/deepmutation/blob/master/LeakyReLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


0it [00:00, ?it/s]

9920512it [00:01, 8906021.12it/s]                            


Extracting MNIST/data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 130059.51it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/data/MNIST/raw


1654784it [00:00, 2220158.11it/s]                            
0it [00:00, ?it/s]

Extracting MNIST/data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/data/MNIST/raw


8192it [00:00, 51317.12it/s]            


Extracting MNIST/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/data/MNIST/raw
Processing...
Done!


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1017
Epoch [1/5], Step [200/600], Loss: 0.1264
Epoch [1/5], Step [300/600], Loss: 0.1207
Epoch [1/5], Step [400/600], Loss: 0.0942
Epoch [1/5], Step [500/600], Loss: 0.1293
Epoch [1/5], Step [600/600], Loss: 0.0338
Epoch [2/5], Step [100/600], Loss: 0.0426
Epoch [2/5], Step [200/600], Loss: 0.0553
Epoch [2/5], Step [300/600], Loss: 0.0227
Epoch [2/5], Step [400/600], Loss: 0.0134
Epoch [2/5], Step [500/600], Loss: 0.0607
Epoch [2/5], Step [600/600], Loss: 0.0691
Epoch [3/5], Step [100/600], Loss: 0.0657
Epoch [3/5], Step [200/600], Loss: 0.0532
Epoch [3/5], Step [300/600], Loss: 0.0071
Epoch [3/5], Step [400/600], Loss: 0.0293
Epoch [3/5], Step [500/600], Loss: 0.0060
Epoch [3/5], Step [600/600], Loss: 0.0650
Epoch [4/5], Step [100/600], Loss: 0.0938
Epoch [4/5], Step [200/600], Loss: 0.0245
Epoch [4/5], Step [300/600], Loss: 0.0536
Epoch [4/5], Step [400/600], Loss: 0.0080
Epoch [4/5], Step [500/600], Loss: 0.0364
Epoch [4/5], Step [600/600], Loss:

In [0]:
path = F"/content/gdrive/My Drive/correct_test_dataset.txt"
correct_test = np.loadtxt(path, delimiter=",", dtype=int)

In [0]:
# Dictionary to record which indices are incorrectly classified
# incorrection_dict[true_label] = what model thinks the class is
incorrection_dict = {}
incorrection_dict[0] = []
incorrection_dict[1] = []
incorrection_dict[2] = []
incorrection_dict[3] = []
incorrection_dict[4] = []
incorrection_dict[5] = []
incorrection_dict[5] = []
incorrection_dict[6] = []
incorrection_dict[7] = []
incorrection_dict[8] = []
incorrection_dict[9] = []

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #only test ones correctly classified by the original model
        for j in range(len(predicted)):
          if idx*batch_size+j in correct_test:
            if labels[j] == predicted[j]:
              correct +=1
            else:
              incorrection_dict[labels[j].item()].append(predicted[j].item())

    total = len(correct_test)
        
    print('Test Accuracy of the model on the {} test images (100% for original model): {} %'.format(total, 100 * correct / total))


Test Accuracy of the model on the 9896 test images (100% for original model): 99.8181083265966 %


In [9]:
print(correct/total)

0.998181083265966


In [10]:
incorrection_dict

{0: [9],
 1: [],
 2: [4, 6],
 3: [2, 5],
 4: [8, 9, 8, 9],
 5: [],
 6: [0, 0],
 7: [2, 1, 1, 1, 1, 2],
 8: [],
 9: [1]}

In [0]:
path =F"/content/gdrive/My Drive/correction_dict.npy" 
correction_dict=np.load(path, allow_pickle=True)

In [12]:
def calc_mutation_score():
  score = 0
  for key, value in incorrection_dict.items():
    count_misclassify = len(value)
    normalizing = len(correction_dict.item().get(key))
    score += count_misclassify/normalizing
  return score
calc_mutation_score()

0.018069560054479007

As expectedly, substituting LeakyRelu for Relu as the mutant achives the lowest mutation affect. Relu and LeakyRelu are identical except for the slight (0.01) negative slope for x < 0. This similarity results in an almost no affect.